<a href="https://colab.research.google.com/github/Harisonm/4aibd-s1-project-ml/blob/yas_branch/4aibd-s1-project-ml/default/apps/notebooks/Rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Simple RNN model for CIFAR-10
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [2]:
K.set_image_dim_ordering('th')

In [3]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

batch_size = 128

In [4]:
# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [5]:
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [6]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [7]:
#reshape X
X_train = numpy.reshape(X_train, (len(X_train),1024, 3))

X_test = numpy.reshape(X_test, (len(X_test),1024, 3))


In [8]:
# Create the model
model = Sequential()
model.add(LSTM(300, input_shape=(1024, 3), 
                    activation='relu', kernel_constraint=maxnorm(3),
                    return_sequences=True
              ))
model.add(Dropout(0.2))
model.add(LSTM(300, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

In [9]:
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1024, 300)         364800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024, 300)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                3010      
Total params: 1,089,010
Trainable params: 1,089,010
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
model_str = "RNN_" + str(epochs) + "_epochs_" + "cifar10" + "categorical_crossentropy"

#model.save("./tensorboard/saved_models" + model_str, True, True)

In [11]:
# Tensorboard callback
tb_callback = TensorBoard(
    log_dir="./tensorboard/logs/" + "RNN" + "_epochs_" + str(epochs) + "RNN_same_relu_softmax")

In [12]:
# Fit the model
# batch_size -> a modifier selon la carte graphique

model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          epochs=epochs,
          batch_size=32)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
